In [1]:
import pandas as pd
import numpy as np
import sys
import os
import gensim
import spacy
import altair as alt
from hyphen import Hyphenator
from gensim.parsing.preprocessing import preprocess_string, STOPWORDS 
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_numeric
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from sklearn.metrics import accuracy_score
import textstat
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
nltk.download('averaged_perceptron_tagger')
cwd = os.getcwd()
from sklearn.svm import SVC
from operator import itemgetter
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
import operator

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\afhar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit           : db08276bc116c438d3fdee492026f8223584c477
python           : 3.7.9.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.19041
machine          : AMD64
processor        : AMD64 Family 23 Model 113 Stepping 0, AuthenticAMD
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : None.None

pandas           : 1.1.3
numpy            : 1.17.0
pytz             : 2020.1
dateutil         : 2.8.1
pip              : 20.2.4
setuptools       : 50.3.1.post20201107
Cython           : 0.29.21
pytest           : 6.1.1
hypothesis       : None
sphinx           : 3.2.1
blosc            : None
feather          : None
xlsxwriter       : 1.3.7
lxml.etree       : 4.6.1
html5lib         : 1.1
pymysql          : None
psycopg2         : None
jinja2           : 2.11.2
IPython          : 7.19.0
pandas_datareader: None
bs4              : 4.9.3
bottleneck       : 1.3.2
fsspe

In [2]:
prep2 = pd.read_pickle('brosko_train_f.pkl')
df_tokenized = pd.read_pickle('df_tokenized')
p_list = ['nouns','verbs', 'Pnouns', 'adjectives', 'adverbs']
for p in p_list:
    col = p + '_%'
    df_tokenized[col] = df_tokenized[p].astype(float)/df_tokenized['num_words'].astype(float)
merged = df_tokenized.merge(prep2, left_index=True, right_index=True, how='inner')

In [3]:
merged.columns

Index(['original_text_x', 'label_x', 'tokens', 'pos', 'nouns', 'verbs',
       'Pnouns', 'adjectives', 'adverbs', 'num_words', 'syls', 'max_syls',
       'avg_syls', 'std_syls', 'flesch_score', 'flesch_grade_lvl',
       'fog_grade_lvl', 'ARI_grade', 'CLI_grade', 'LWF_grade',
       'Dale-Chall_score', 'combined_grade', 'nouns_%', 'verbs_%', 'Pnouns_%',
       'adjectives_%', 'adverbs_%', 'index', 'original_text_y', 'label_y',
       'clean_text', 'word_count', 'clean_text_no_stop', 'word_count_no_stop',
       'Nsyll', 'AoA_Kup_lem', 'Perc_known_lem'],
      dtype='object')

In [4]:
#define the features and scale the data
cols = ['nouns', 'verbs',
        'Pnouns', 'adjectives', 'adverbs', 'num_words', 'max_syls',
        'nouns_%', 'verbs_%', 'Pnouns_%','adjectives_%', 'adverbs_%',
        'avg_syls', 'std_syls', 'flesch_score', 'flesch_grade_lvl',
        'fog_grade_lvl', 'ARI_grade', 'CLI_grade', 'LWF_grade',
        'Dale-Chall_score','word_count_no_stop','Nsyll','AoA_Kup_lem','Perc_known_lem']
X_train = merged[cols]
#X_train = merged[['flesch_score', 'flesch_grade_lvl',
#       'fog_grade_lvl', 'ARI_grade', 'CLI_grade', 'LWF_grade',
#       'Dale-Chall_score']]
X_train = X_train.replace(np.nan, 0)
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
y_train = merged['label_x'].values

In [5]:
#Split the data
X= X_train
y = y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train_df = pd.DataFrame(X_train, columns = cols)
X_test_df = pd.DataFrame(X_test, columns = cols)

In [10]:
#Train the model
estimator_SVM = SGDClassifier()
params = {
    'loss':['hinge'],
    'penalty': ['l2','l1'],
    'alpha': [0.0001,0.001,0.1],

}
#use grid search for tuning
grid_search_SVM = GridSearchCV(
    estimator=estimator_SVM,
    param_grid=params,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5,
    verbose=3
)


In [11]:
#develop a couple different feature sets
X_train_1 = X_train_df.values #full
X_train_2 = X_train_df[['flesch_score', 'flesch_grade_lvl','fog_grade_lvl', 'ARI_grade', 'CLI_grade', 'LWF_grade','Dale-Chall_score']].values #just other scorers
X_train_3 = X_train_df[['num_words', 'max_syls','nouns_%', 'verbs_%', 'Pnouns_%','adjectives_%', 'adverbs_%','avg_syls', 'std_syls','AoA_Kup_lem','Perc_known_lem']].values #just text stats
X_train_4 = X_train_df[['AoA_Kup_lem','Perc_known_lem','word_count_no_stop' ]].values #basic model

X_test_1 = X_test_df.values #full
X_test_2 = X_test_df[['flesch_score', 'flesch_grade_lvl','fog_grade_lvl', 'ARI_grade', 'CLI_grade', 'LWF_grade','Dale-Chall_score']].values #just other scorers
X_test_3 = X_test_df[['num_words', 'max_syls','nouns_%', 'verbs_%', 'Pnouns_%','adjectives_%', 'adverbs_%','avg_syls', 'std_syls','AoA_Kup_lem','Perc_known_lem']].values #just text stats
X_test_4 = X_test_df[['AoA_Kup_lem','Perc_known_lem','word_count_no_stop' ]].values #basic model

#Do the search
SVM_1=grid_search_SVM.fit(X_train_1, y_train) #full feature list
y_pred_SVM =SVM_1.predict(X_test_1)
print('1 done')
SVM_2=grid_search_SVM.fit(X_train_2, y_train) #just other scoring methodsI w
y_pred_SVM2 =SVM_2.predict(X_test_2)
print('2 done')
SVM_3=grid_search_SVM.fit(X_train_3, y_train) #just text stats
y_pred_SVM3 =SVM_3.predict(X_test_3)
print('3 done')
SVM_4=grid_search_SVM.fit(X_train_4, y_train) #basic model, probably likes starbucks and uggs
y_pred_SVM4 =SVM_4.predict(X_test_4)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  30 | elapsed:   10.0s remaining:   20.1s
[Parallel(n_jobs=-1)]: Done  21 out of  30 | elapsed:   11.7s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   13.3s finished


1 done
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  30 | elapsed:    3.2s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done  21 out of  30 | elapsed:    4.2s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    5.1s finished


2 done
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  30 | elapsed:    3.8s remaining:    7.8s
[Parallel(n_jobs=-1)]: Done  21 out of  30 | elapsed:    5.3s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    5.7s finished


3 done
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  30 | elapsed:    2.8s remaining:    5.7s
[Parallel(n_jobs=-1)]: Done  21 out of  30 | elapsed:    3.9s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    4.3s finished


In [12]:
#Parameter setting that gave the best results on the hold out data.
print(grid_search_SVM.best_params_ ) 
#Mean cross-validated score of the best_estimator
print('Best Score - SVM:', grid_search_SVM.best_score_ )

{'alpha': 0.001, 'loss': 'hinge', 'penalty': 'l1'}
Best Score - SVM: 0.6270547390337075


In [10]:
print('Best Score - KNN:', KNN_1.best_score_ )

Best Score - KNN: 0.6684393827485577


In [14]:
#check with the hold out data
print('Accuracy Score - SVM - 1:', metrics.accuracy_score(y_test, y_pred_SVM)) 
print('Accuracy Score - SVM - 2:', metrics.accuracy_score(y_test, y_pred_SVM2)) 
print('Accuracy Score - SVM - 3:', metrics.accuracy_score(y_test, y_pred_SVM3)) 
print('Accuracy Score - SVM - 4:', metrics.accuracy_score(y_test, y_pred_SVM4)) 

Accuracy Score - SVM - 1: 0.6622798726133174
Accuracy Score - SVM - 2: 0.645949365247866
Accuracy Score - SVM - 3: 0.6314293192955923
Accuracy Score - SVM - 4: 0.6239402620442945


In [36]:
#prepare the actual test data for submission
submit_df = pd.read_pickle('test_data.pkl')
p_list = ['nouns','verbs', 'Pnouns', 'adjectives', 'adverbs']
for p in p_list:
    col = p + '_%'
    submit_df[col] = submit_df[p].astype(float)/submit_df['num_words'].astype(float)
submit_df = submit_df.replace(np.nan, 0)
submit_s = scaler.transform(submit_df[cols])
submit = pd.DataFrame(submit_s, columns = cols)
X_sub_1 = submit.values #full
X_sub_2 = submit[['flesch_score', 'flesch_grade_lvl','fog_grade_lvl', 'ARI_grade', 'CLI_grade', 'LWF_grade','Dale-Chall_score']].values #just other scorers
X_sub_3 = submit[['num_words', 'max_syls','nouns_%', 'verbs_%', 'Pnouns_%','adjectives_%', 'adverbs_%','avg_syls', 'std_syls','AoA_Kup_lem','Perc_known_lem']].values #just text stats
X_sub_4 = submit[['AoA_Kup_lem','Perc_known_lem','word_count_no_stop' ]].values #basic model



In [40]:
#train the best model
knn_f = KNeighborsClassifier(leaf_size= 20, metric= 'chebyshev', n_neighbors=1000, p= 1, weights= 'distance') #start with square root of N, to add a loop later
knn_f.fit(X_train_1, y_train)
y_pred = knn_f.predict(X_sub_1)
df = pd.DataFrame()
df['id'] =submit_df.index
df['label'] = y_pred
df.to_csv('knn_textscores_only.csv',index = False)
# 0.71537 on kaggle! passing

In [44]:
#train the best "small" model
knn_f_s = KNeighborsClassifier(leaf_size= 20, metric= 'chebyshev', n_neighbors=1000, p= 1, weights= 'distance') #start with square root of N, to add a loop later
knn_f_s.fit(X_train_2, y_train)
y_pred = knn_f_s.predict(X_sub_2)
df = pd.DataFrame()
df['id'] =submit_df.index
df['label'] = y_pred
df.to_csv('knn_textscores_only.csv',index = False)
#0.70100, barely worse

In [37]:
y_pred = KNN_1.predict(X_sub_4)
df = pd.DataFrame()
df['id'] =submit_df.index
df['label'] = y_pred
df.to_csv('knn_textscores_only.csv',index = False)

In [43]:
X_sub_2.shape

(119092, 7)